<a href="https://colab.research.google.com/github/TAbdullah-T/T5-SAD/blob/main/Deep%20Learning/Projects/5_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Dataset

In [ ]:
!git clone https://github.com/TariqAlhathloul/GTS_CNN.git
!pip install keras-tuner --upgrade

Cloning into 'GTS_CNN'...
remote: Enumerating objects: 51933, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 51933 (delta 0), reused 2 (delta 0), pack-reused 51930 (from 1)
Receiving objects: 100% (51933/51933), 299.46 MiB | 15.29 MiB/s, done.
Updating files: 100% (51890/51890), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
#import tensorflow and keras
import tensorflow as tf
import keras
from keras import models, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import kerastuner as kt
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'kerastuner'

In [ ]:
train_csv_path = '/content/GTS_CNN/DataSet/Train.csv'
test_csv_path = '/content/GTS_CNN/DataSet/Test.csv'

base_path = '/content/GTS_CNN/DataSet/'

img_width, img_height = 30, 30

def load_and_resize_image(filepath):
    full_path = os.path.join(base_path, filepath)
    image = Image.open(full_path).convert('RGB')
    image = image.resize((img_width, img_height))
    return np.array(image)

# Load CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Load training data
X_train = []
y_train = []
for _, row in train_df.iterrows():
    image_path = row['Path']
    class_id = row['ClassId']
    X_train.append(load_and_resize_image(image_path))
    y_train.append(class_id)

# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Load test data
X_test = []
y_test = []
for _, row in test_df.iterrows():
    image_path = row['Path']
    class_id = row['ClassId']
    X_test.append(load_and_resize_image(image_path))
    y_test.append(class_id)
# Convert to numpy array
X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'X_test shape: {y_test.shape}')

## Data Agumentation

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = 0.2,
)
datagen.fit(X_train)

# Model 1

In [ ]:
model = Sequential([
    #Block 1 : 32 units
    Conv2D(32,(3,3), activation='relu',padding='same', input_shape=(30,30,3)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(32,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),

    #Block 2 : 64 units
    Conv2D(32,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(32,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),

    # #Block 3 : 64 units
    # Conv2D(32,(3,3), activation='relu', padding='same'),
    # BatchNormalization(),
    # Dropout(0.5),
    # Conv2D(32,(3,3), activation='relu', padding='same'),
    # BatchNormalization(),
    # Dropout(0.5),
    # MaxPooling2D(2,2),

    # Classifiction head
    Flatten(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(43, activation='softmax')
])

model.compile(optimizer= 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=0)

In [ ]:

early_stoping = EarlyStopping(monitor = 'val_accuracy', patience = 5 )
history = model.fit(datagen.flow(X_train,y_train,batch_size=64),
                    batch_size= 1000,
                    epochs = 10,
                    validation_data = (X_val,y_val),
                    callbacks=[early_stoping],
                    verbose=2)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


393/393 - 39s - 100ms/step - accuracy: 0.2245 - loss: 2.9216 - val_accuracy: 0.1715 - val_loss: 17.6871
Epoch 2/10
393/393 - 22s - 55ms/step - accuracy: 0.4647 - loss: 1.7769 - val_accuracy: 0.1603 - val_loss: 65.3490
Epoch 3/10
393/393 - 20s - 51ms/step - accuracy: 0.5926 - loss: 1.2985 - val_accuracy: 0.1881 - val_loss: 83.8267
Epoch 4/10
393/393 - 21s - 53ms/step - accuracy: 0.6697 - loss: 1.0318 - val_accuracy: 0.2134 - val_loss: 60.9980
Epoch 5/10
393/393 - 20s - 50ms/step - accuracy: 0.7186 - loss: 0.8637 - val_accuracy: 0.1742 - val_loss: 64.0538
Epoch 6/10
393/393 - 17s - 44ms/step - accuracy: 0.7534 - loss: 0.7434 - val_accuracy: 0.1654 - val_loss: 91.3934
Epoch 7/10
393/393 - 19s - 49ms/step - accuracy: 0.7783 - loss: 0.6660 - val_accuracy: 0.2243 - val_loss: 60.5056
Epoch 8/10
393/393 - 16s - 41ms/step - accuracy: 0.8010 - loss: 0.5946 - val_accuracy: 0.2499 - val_loss: 53.6513
Epoch 9/10
393/393 - 20s - 52ms/step - accuracy: 0.8153 - loss: 0.5538 - val_accuracy: 0.2246 - va

# Model 2

In [ ]:
X_train_subset = X_train[:int(len(X_train_) * 0.15)]
y_train_subset = y_train[:int(len(X_train_) * 0.15)]
X_val_subset = X_val[:int(len(X_val) * 0.13)]
y_val_subset = y_val[:int(len(X_val) * 0.13)]
print(X_train_subset.shape, y_train_subset.shape, X_val_subset.shape)

In [ ]:
def model_builder(hp):

    filter_values_1 = hp.Choice('Number of units B1', [8 , 16, 32 ])
    filter_values_2 = hp.Choice('Number of units B2', [16, 32, 64 ])
    filter_values_3 = hp.Choice('Number of units B3', [32, 64, 128])

    activation_choice = hp.Choice('activation_function', ['relu', 'leaky_relu', 'tanh'])

    model = Sequential([
    #Block 1 : 32 units
    Conv2D(filter_values_1,(3,3), activation=activation_choice,padding='same', input_shape=(120,120,3),kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filter_values_1,(3,3), activation=activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),

    #Block 2 : 64 units
    Conv2D(filter_values_2,(3,3), activation= activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filter_values_2,(3,3), activation=activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),


    # #Block 3 : 64 units
    Conv2D(filter_values_3,(3,3), activation= activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),

    # Classifiction head
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(43, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def build_datagen(hp):
    return ImageDataGenerator(
        rotation_range=hp.Int('rotation_range', min_value=0, max_value=90, step=10),
        width_shift_range=hp.Float('width_shift_range', min_value=0.0, max_value=0.3, step=0.05),
        height_shift_range=hp.Float('height_shift_range', min_value=0.0, max_value=0.3, step=0.05),
        zoom_range=hp.Float('zoom_range', min_value=0.0, max_value=0.3, step=0.05),
        horizontal_flip=hp.Boolean('horizontal_flip'),
        rescale=1/255
    )

def train_test_model(hp):
    # Create the data generator with hyperparameters
    datagen = build_datagen(hp)
    datagen.fit(X_train_subset)
    model = model_builder(hp)

    model.fit(
        datagen.flow(X_train_subset, y_train_subset, batch_size=64),
        validation_data=datagen.flow(X_val_subset, y_val_subset, batch_size=64),
        epochs=15,
        verbose=0)
    return model

# Define and initialize the tuner
tuner = kt.RandomSearch(
    train_test_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='auto_tune_aug',
    project_name='augmentation_tuning'
)

tuner.search()

## Choosing the Best Model

In [ ]:
def model_builder(filter_values_1, filter_values_2, filter_values_3, activation_choice):

    model = Sequential([
            #Block 1 : 32 units
    Conv2D(filter_values_1,(3,3), activation=activation_choice,padding='same', input_shape=(120,120,3),kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filter_values_1,(3,3), activation=activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),

    #Block 2 : 64 units
    Conv2D(filter_values_2,(3,3), activation= activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filter_values_2,(3,3), activation=activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),


    # #Block 3 : 64 units
    Conv2D(filter_values_3,(3,3), activation= activation_choice, padding='same',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    BatchNormalization(),
    Dropout(0.5),

    # Classifiction head
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(43, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def build_datagen():
  return ImageDataGenerator(
          rotation_range=60,
          width_shift_range=0.2,
          height_shift_range=0.1,
          zoom_range= 0.1,
          horizontal_flip=True,
          rescale=1/255)


    # Create the data generator with hyperparameters
datagen = build_datagen()
datagen.fit(X_train_subset)
model = model_builder(16,64,128, 'leaky_relu')

model.fit(
    datagen.flow(x_for_train_subset, y_for_train_subset, batch_size=64),
    validation_data=(X_for_val_subset, y_for_val_subset),
    batch_size=100,
    epochs=100,
    verbose=1)
